In [5]:

import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


In [15]:
session = pd.read_csv('data/sessions.csv')
session

,hashedEmail,start_time,end_time,original_start_time,original_end_time
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,30/06/2024 18:12,30/06/2024 18:24,1.719770e+12,1.719770e+12
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,17/06/2024 23:33,17/06/2024 23:46,1.718670e+12,1.718670e+12
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,25/07/2024 17:34,25/07/2024 17:57,1.721930e+12,1.721930e+12
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,25/07/2024 03:22,25/07/2024 03:58,1.721880e+12,1.721880e+12
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,25/05/2024 16:01,25/05/2024 16:12,1.716650e+12,1.716650e+12
...,...,...,...,...,...
1530,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,10/05/2024 23:01,10/05/2024 23:07,1.715380e+12,1.715380e+12
1531,7a4686586d290c67179275c7c3dfb4ea02f4d317d9ee0e...,01/07/2024 04:08,01/07/2024 04:19,1.719810e+12,1.719810e+12
1532,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,28/07/2024 15:36,28/07/2024 15:57,1.722180e+12,1.722180e+12
1533,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,25/07/2024 06:15,25/07/2024 06:22,1.721890e+12,1.721890e+12


In [7]:
DATA SET DESCRIPTION:
    I will be using the session_url data set only. Using this dataset will provide me with the necessary data on the number of simultaneous
    players online all throughout the day. I will not be utilizing the players data set because it's data regarding each individual player is 
    irrelevant in identifying the number of licenses to purchase to properly accommodate the player base.
        The session data set contains a total of 1534 observations with 5 original variables: hashedEmail (ID), start_time(Date+Time session began),
        end_time(Date+time session ended), original_start_time, and original_end_time. 
        

QUESTION: 
#2: what time windows are most likely to have large number of simultaneous players and number of seats to appropriately accomodate the player base at certain times 
in different days of the week.

EXPLORATORY DATA AND VISUALIZATION:
I will be dropping the hashedEmail, original_start_time and original_end_time variable as it is irrelevant to my goal of identifying the number
of simultaneous players at a time. To wrangle the data, I will .str.split the start and end time varibales to separate Date vs Time. I will then convert
the newly separated time into a numeric float data type in order to calculate the session length, which will be a parameter set to only recognize those
that have played/was online for at least 10 minutes and to filter out the data. From there I plan on identifying the Dates to be days of the week
using the groupby and pd.to.datetime functions to properly arrange the data. Lastly in order to visualize it, I will bin it by maxbin= 24 to set it 
to a 24 hr time plot and average the mean count of how many were online at a certain hour and facet it by the day of the week to have 7 separate plots.

From this visualisation, I recognize that there is a significant trend throughout the week where players tend to play near the extreme ends of the day
(near 0300-0600 and 1600-2000). 
    
METHODS AND PLANNING
I plan on splitting the data 65% for training data and utilizing linear regression model to predict the number of licenses by the hours by the day of the week
and since the extreme ends of the days appear to be somewhat linear enough for me to justify the usage of linear regression albeit with slight adjustments
to how the predictor variable is wrangled. overall I believe that the utilization of linear regression would be an appropriate model to encompass
this data set to identify the number of licenses needed by set hours in the days of the week.



SyntaxError: unterminated string literal (detected at line 3) (3886002495.py, line 3)

In [36]:
session = session[['start_time', 'end_time']] #selected only useful starting variables
session[['start_date', 'start_time_hr']] = session['start_time'].str.split( " ", expand=True) #
session[['end_date', 'end_time_hr']] = session['end_time'].str.split( " ", expand=True) 

session_1 = session
session_1['end_time_hr'] = session_1['end_time_hr'].str.replace(":","")
session_1['start_time_hr'] = session_1['start_time_hr'].str.replace(":","")
session_1['start_time_hr'] = pd.to_numeric(session_1['start_time_hr'], errors='coerce')     
session_1['end_time_hr'] = pd.to_numeric(session_1['end_time_hr'], errors='coerce')     

session_up = session.assign( ses_length = session_1['end_time_hr'] - session_1['start_time_hr'])
session_updated = session_up[session_up['ses_length'] > 10].reset_index()
session_updated

/tmp/ipykernel_1581/2155008333.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  session[['start_date', 'start_time_hr']] = session['start_time'].str.split( " ", expand=True) #
/tmp/ipykernel_1581/2155008333.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  session[['start_date', 'start_time_hr']] = session['start_time'].str.split( " ", expand=True) #
/tmp/ipykernel_1581/2155008333.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,index,start_time,end_time,start_date,start_time_hr,end_date,end_time_hr,ses_length
0,0,30/06/2024 18:12,30/06/2024 18:24,30/06/2024,1812,30/06/2024,1824.0,12.0
1,1,17/06/2024 23:33,17/06/2024 23:46,17/06/2024,2333,17/06/2024,2346.0,13.0
2,2,25/07/2024 17:34,25/07/2024 17:57,25/07/2024,1734,25/07/2024,1757.0,23.0
3,3,25/07/2024 03:22,25/07/2024 03:58,25/07/2024,322,25/07/2024,358.0,36.0
4,4,25/05/2024 16:01,25/05/2024 16:12,25/05/2024,1601,25/05/2024,1612.0,11.0
...,...,...,...,...,...,...,...,...
1040,1527,05/07/2024 04:29,05/07/2024 04:45,05/07/2024,429,05/07/2024,445.0,16.0
1041,1528,22/06/2024 02:30,22/06/2024 04:22,22/06/2024,230,22/06/2024,422.0,192.0
1042,1531,01/07/2024 04:08,01/07/2024 04:19,01/07/2024,408,01/07/2024,419.0,11.0
1043,1532,28/07/2024 15:36,28/07/2024 15:57,28/07/2024,1536,28/07/2024,1557.0,21.0


In [96]:

session_updated['start_date'] = pd.to_datetime(session_updated['start_date'], format='%d/%m/%Y')

session_updated['Day'] = session_updated['start_date'].dt.day_name()

session_updated['end_time_hr'] = pd.to_datetime(session_updated['end_time_hr'], format='%H%M')
session_updated['Hour'] = session_updated['end_time_hr'].dt.hour  # identify only the hour portion

session_count = session_updated.groupby(['Day', 'Hour']).size().reset_index(name='Count')
# pd.set_option('display.max_rows', None)
# print(session_count)

session_plot = alt.Chart(session_counted).mark_bar().encode(
    x=alt.X('Hour').bin(maxbins=24).title("Time (24hrs)").scale(zero=True),  # X axis as hour, binned
    y= alt.Y('mean(Count)').title('Mean Count'),  # Y axis is the mean count of data points
).facet('Day')


session_plot

alt.FacetChart(...)